# Distilling Knowledge in Multiple Students Using Generative Models

In [1]:
# %tensorflow_version 1.x
# !pip install --upgrade opencv-python==3.4.2.17
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils



In [2]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
# convert y_train and y_test to categorical binary values 
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(teacher.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [3]:
# Train the teacher model as usual
epochs = 20
batch_size = 256
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))


Epoch 1/20
188/188 [==============================] - 4s 11ms/step - loss: 0.8719 - accuracy: 0.6807 - val_loss: 0.3312 - val_accuracy: 0.8777
Epoch 2/20
188/188 [==============================] - 2s 10ms/step - loss: 0.3500 - accuracy: 0.8756 - val_loss: 0.3218 - val_accuracy: 0.8813
Epoch 3/20
188/188 [==============================] - 2s 11ms/step - loss: 0.2820 - accuracy: 0.8993 - val_loss: 0.2854 - val_accuracy: 0.8932
Epoch 4/20
188/188 [==============================] - 2s 11ms/step - loss: 0.2463 - accuracy: 0.9121 - val_loss: 0.2439 - val_accuracy: 0.9113
Epoch 5/20
188/188 [==============================] - 2s 9ms/step - loss: 0.2072 - accuracy: 0.9225 - val_loss: 0.2246 - val_accuracy: 0.9153
Epoch 6/20
188/188 [==============================] - 2s 9ms/step - loss: 0.1813 - accuracy: 0.9339 - val_loss: 0.2160 - val_accuracy: 0.9222
Epoch 7/20
188/188 [==============================] - 2s 9ms/step - loss: 0.1549 - accuracy: 0.9427 - val_loss: 0.2151 - val_accuracy: 0.9221
Ep

In [7]:
teacher.save_weights("Teacher_FMNIST_92.h5")

In [6]:
Y_test = np_utils.to_categorical(y_test, nb_classes)
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
X_test /= 255


teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.3516 - accuracy: 0.9239


[0.351641982793808, 0.9239000082015991]

In [8]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)


# 2 Students case
# ---------------------------------------------
s1Train=train_dense[:,:128]
s2Train=train_dense[:,128:]


## GANs' Training

In [9]:
# import np.random import random
BATCH_SIZE=32
def smooth_real_labels(y):
    return y - 0.3+(np.random.random(y.shape)*0.5)
def smooth_fake_labels(y):
    return y + (0.3 * np.random.random(y.shape))
def build_gan(gen,disc): 
    disc.trainable = False
    input= Input(shape=input_shape)
    output = gen(input)
    output2= disc(output)
    gan=Model(input,output2)

    gan.compile(Adam(lr=0.0002),loss=['binary_crossentropy','mse'],metrics=['accuracy'])

    return gan

In [10]:
def build_sdiscriminator():
    
    input2 = Input(shape=(128,),name='input')
    inp=Dense(64,use_bias=False)(input2)

    leaky_relu = LeakyReLU(alpha=0.2)(inp)
    
    conv3 = Dense(128)(leaky_relu)
    b_n = BatchNormalization()(conv3)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(256)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(512)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(1024)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)

    dense = Dense(1,activation='sigmoid',name='dense')(leaky_relu)
    output2=Dense(128)(leaky_relu)
    disc = Model(input2,[dense,output2])          
    disc.compile(optd,loss=['binary_crossentropy','mse'],metrics=['accuracy'])

    return disc

In [14]:
optd = Adam(lr=0.0002)
opt = Adam(lr=0.0002)

In [17]:
def build_sgenerator(name):

    student1 = Sequential()
    student1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(16, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(128,name='req'+name))

    student1.compile(opt,loss='mse',metrics=['accuracy'])
    student1.summary()
    return student1



In [18]:
def training(generator,discriminator,gan,features,epo=20):
    BATCH_SIZE = 128
    discriminator.trainable = True
    total_size = X_train.shape[0]
    indices = np.arange(0,total_size ,BATCH_SIZE)
    all_disc_loss = []
    all_gen_loss = []
    all_class_loss=[]
    if total_size % BATCH_SIZE:
        indices = indices[:-1]
    for e in range(epo):
        
        progress_bar = Progbar(target=len(indices))
        np.random.shuffle(indices)
        epoch_gen_loss = []
        epoch_disc_loss = []
        epoch_class_loss= []
        for i,index in enumerate(indices):
        
            inputs=X_train[index:index+BATCH_SIZE]
            strain = features[index:index+BATCH_SIZE]
            y_real = np.ones((BATCH_SIZE,1))
            y_fake = np.zeros((BATCH_SIZE,1))

            #Generator Training
            fake_images = generator.predict_on_batch(inputs)

            #Disrciminator Training
            disc_real_loss1,_,disc_real_loss2,_,_= discriminator.train_on_batch(strain,[y_real,strain])
            disc_fake_loss1,_,disc_fake_loss2,_,_= discriminator.train_on_batch(fake_images,[y_fake,strain])

            #Gans Training
            discriminator.trainable = False
            gan_loss,_,gan_loss2,_,_ = gan.train_on_batch(inputs, [y_real,strain])

            discriminator.trainable = True

            disc_loss = (disc_fake_loss1 + disc_real_loss1)/2
            epoch_disc_loss.append(disc_loss)
            progress_bar.update(i+1)

            epoch_gen_loss.append((gan_loss))

        avg_epoch_disc_loss = np.array(epoch_disc_loss).mean()
        avg_epoch_gen_loss = np.array(epoch_gen_loss).mean()
        all_disc_loss.append(avg_epoch_disc_loss)
        all_gen_loss.append(avg_epoch_gen_loss)
        print("Epoch: %d | Discriminator Loss: %f | Generator Loss: %f | " % (e+1,avg_epoch_disc_loss,avg_epoch_gen_loss))

    return generator

In [19]:
discriminator1 = build_sdiscriminator()
discriminator2 = build_sdiscriminator()

In [20]:
s1=build_sgenerator("s1")
s2=build_sgenerator('s2')
# s3=build_sgenerator("s3")
# s4=build_sgenerator('s4')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Conv2D)                  (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 10, 10, 16)        4624      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 8, 16)          2320      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 4, 4, 16)         

In [21]:
gan1 = build_gan(s1,discriminator1)
gan2 = build_gan(s2,discriminator2)


In [22]:
s1 = training(s1,discriminator1,gan1,s1Train,epo=55)
s2 = training(s2,discriminator2,gan2,s2Train,epo=55)


375/375 [==============================] - 15s 35ms/step
Epoch: 1 | Discriminator Loss: 0.484974 | Generator Loss: 1.522924 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 2 | Discriminator Loss: 0.232264 | Generator Loss: 1.050430 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 3 | Discriminator Loss: 0.185770 | Generator Loss: 0.835174 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 4 | Discriminator Loss: 0.165592 | Generator Loss: 0.767950 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 5 | Discriminator Loss: 0.154636 | Generator Loss: 0.755518 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 6 | Discriminator Loss: 0.146981 | Generator Loss: 0.735421 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 7 | Discriminator Loss: 0.141064 | Generator Loss: 0.720830 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 8 | Discriminator Loss: 0.137847 | Generator Lo

375/375 [==============================] - 13s 36ms/step
Epoch: 10 | Discriminator Loss: 0.143047 | Generator Loss: 0.815172 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 11 | Discriminator Loss: 0.139876 | Generator Loss: 0.755555 | 
375/375 [==============================] - 14s 36ms/step
Epoch: 12 | Discriminator Loss: 0.137450 | Generator Loss: 0.732512 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 13 | Discriminator Loss: 0.134187 | Generator Loss: 0.702328 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 14 | Discriminator Loss: 0.131984 | Generator Loss: 0.677313 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 15 | Discriminator Loss: 0.129665 | Generator Loss: 0.672877 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 16 | Discriminator Loss: 0.128412 | Generator Loss: 0.663011 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 17 | Discriminator Loss: 0.127494 | Gene

### **2 Students**

In [23]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
output=tensorflow.keras.layers.concatenate([o1,o2])

output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)

mm2=Model([s1.get_layer("s1").input,s2.get_layer("s2").input], output3)
my_weights=teacher.get_layer('dense_2').get_weights()
mm2.get_layer('d1').set_weights(my_weights)

In [24]:
for l in mm2.layers[:len(mm2.layers)-1]:
    l.trainable=False

mm2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [25]:
batch_size = 128
mm2_history=mm2.fit([X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=30,
          verbose=1, validation_data=([X_val,X_val], Y_val))

Epoch 1/30
375/375 [==============================] - 2s 4ms/step - loss: 5.6394 - accuracy: 0.3523 - val_loss: 0.5095 - val_accuracy: 0.8277
Epoch 2/30
375/375 [==============================] - 2s 5ms/step - loss: 1.2643 - accuracy: 0.6552 - val_loss: 0.5004 - val_accuracy: 0.8332
Epoch 3/30
375/375 [==============================] - 2s 5ms/step - loss: 0.8876 - accuracy: 0.7046 - val_loss: 0.5012 - val_accuracy: 0.8329
Epoch 4/30
375/375 [==============================] - 3s 9ms/step - loss: 0.7946 - accuracy: 0.7241 - val_loss: 0.4948 - val_accuracy: 0.8307
Epoch 5/30
375/375 [==============================] - 2s 5ms/step - loss: 0.7324 - accuracy: 0.7362 - val_loss: 0.5033 - val_accuracy: 0.8256
Epoch 6/30
375/375 [==============================] - 3s 7ms/step - loss: 0.7147 - accuracy: 0.7353 - val_loss: 0.4904 - val_accuracy: 0.8267
Epoch 7/30
375/375 [==============================] - 3s 8ms/step - loss: 0.6974 - accuracy: 0.7429 - val_loss: 0.4987 - val_accuracy: 0.8313
Epoch 

In [26]:
loss, acc = mm2.evaluate([X_test,X_test], Y_test, verbose=1)
loss, acc 

313/313 [==============================] - 2s 5ms/step - loss: 0.5186 - accuracy: 0.8217


(0.5186209678649902, 0.8216999769210815)